In [3]:
# hyperparamter tuning using keras tuner
# https://www.tensorflow.org/tutorials/keras/keras_tuner
import tensorflow as tf
import keras
import kerastuner as kt
import segmentation_models as sm
import numpy as np
import os
import pydensecrf.densecrf as dcrf
import random
import imageio
import time
from hyperopt.pyll import scope
from hyperopt import hp, tpe, Trials, fmin, STATUS_OK
from pydensecrf import utils
from tqdm import tqdm
from PIL import Image
from glob import glob

Using TensorFlow backend.


Segmentation Models: using `keras` framework.


In [4]:
global HEIGHT
global WIDTH

HEIGHT = 256
WIDTH = 256

In [5]:
class TqdmUpTo(tqdm):
    def update_to(self, b=1, bsize=1, tsize=None):
        if tsize is not None:
            self.total = tsize
        self.update(b * bsize - self.n)

In [6]:
# image = Image.open("../dataset/testing/images/labels/mask_binary_28_47.tif")
# image = np.array(image)
# np.unique(image)
# image
# #imageio.imread("../dataset/tuning/annotations/annotation_0.jpg").shape

In [25]:
def gen_pred_probs(img_dir, out_dir, weight_file):
# generate prediction probabilites (.npy) for every image(.jpg) in the dataset
# saved as numpy arrays(.npy)
# expects all images in img_dir to be named "image_0000.jpg" (code is specific to image)
# resulting prob files -> "out_dir/pred-probs_0000.npy"
# will clear probabilites directory before writing

    #Listing GPU info
    gpus = tf.config.experimental.list_physical_devices('GPU')
    if gpus:
       try:
           for gpu in gpus:
               tf.config.experimental.set_memory_growth(gpu, True)
           logical_gpus = tf.config.experimental.list_logical_devices('GPU')
           print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
       except RuntimeError as e:
           print(e)
    
    model = sm.Unet(
        'vgg16', 
        #backbone,
        input_shape=(HEIGHT, WIDTH, 3), 
        encoder_weights='imagenet', 
        weights=weight_file,
        encoder_freeze=True,    # only training decoder network
        classes= 2, 
        activation='softmax'
    )

    # Might be unnecessary
    model.compile(
        'Adam', 
        loss=sm.losses.DiceLoss(),
        metrics=[sm.metrics.iou_score]
    )
    
    if os.path.exists(out_dir):
        os.system("rm " + out_dir + "/*") # delete all files in /out_dir
    else:
        print(f"out_dir:{out_dir} does not exist")
        
        
    if os.path.exists(img_dir):
        full_dataset = glob(img_dir + "*.jpg") # glob all jpgs
    else:
        print(f"img_dir:{img_dir} does not exist")
        os.exit()
        
    
    for img_fp in tqdm(full_dataset):
        img = np.asarray(Image.open(img_fp)) / 255.0 # assuming model uses normalized imgs
        img = img[np.newaxis, ...] # needs (batch_size, h, w, channels)
        pred_probs = model.predict(img)[0] # shape: -> (256, 256, 2)
        pred_probs = 1 - pred_probs # classes do be flipped
        pred_probs = pred_probs * 255
        pred_probs = np.array(pred_probs).astype('uint8')  # should be binary (0, 255) values only
        probs_fp = img_fp.replace(img_dir, out_dir)
        probs_fp = probs_fp.replace("image", "pred-probs")
        probs_fp = probs_fp.replace(".jpg", ".npy")
        np.save(probs_fp, pred_probs)

In [8]:
def parse_image(img_fp: str) -> dict:
    """Load an image, its annotation (true_mask), and predicted probabilites, returning
    a dictionary. Assumes images are in "tuning/images/", annots are in "tuning/annotations/"
    and pred probabilites are in "tuning/probs/"

    Parameters
    ----------
    img_path : str
        Image (not the mask) location.

    Returns
    -------
    dict
        Dictionary mapping an image, annotation, and predicted probabilites.
    """
    img = np.asarray(Image.open(img_fp))
    img = np.reshape(img, (256, 256, 3))

    
    tmask_fp = img_fp.replace("images", "annotations")
    tmask_fp = img_fp.replace("image", "annotation")
    true_mask = np.asarray(Image.open(tmask_fp))
    true_mask = np.reshape(true_mask, (256, 256, 3))

    probs_fp = img_fp.replace("images", "probs")
    probs_fp = probs_fp.replace("image", "pred-probs")
    probs_fp = probs_fp.replace(".jpg", ".npy")
    pred_probs = np.load(probs_fp) # these ones are already .npy
    pred_probs = np.reshape(pred_probs, (256, 256, 2)) # probabilites of 2 classes, passed to densecrf

    return {'image': img, 'true_mask': true_mask, 'pred_probs': pred_probs} # all Numpy Arrays

In [9]:
def compile_dataset(sample_percentage: float):
# Assume all data is in ../dataset/tuning/
# Load datasets AFTER calling unet_funcs_tuning.tune_setup()
    """
    loads test images, ground truth masks, and predicted probabilites into dataset, 
    sample_percentage is a float < 1 that represents how much of the full dataset will be used
    in tuning
    
    Returns: tuple
    -------
        
        (test_dataset, DATASET_SIZE): (iter, int)
        
        # test_dataset: a "map" iter with dicts of three np.arrays
        
    """
    
    assert sample_percentage < 1
    
    SEED = 42
    BATCH_SIZE = 16
    BUFFER_SIZE = 1000
    
    tuning_data = "../dataset/tuning/"
    
    gpus = tf.config.experimental.list_physical_devices('GPU')
    if gpus:
        try:
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
            logical_gpus = tf.config.experimental.list_logical_devices('GPU')
            print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
        except RuntimeError as e:
            print(e)
            
    full_dataset = glob(tuning_data + "images/*.jpg") # get tiles, full dataset ~106560 imgs
    DATASET_SIZE = len(full_dataset)
    print(f"The Tuning FULL Dataset contains {DATASET_SIZE} images.")

    SAMPLE_SIZE = int(DATASET_SIZE * sample_percentage) # how many dataset entries to use in tuning
    print(f"Taking sample of {SAMPLE_SIZE} from main dataset..")
    test_dataset = random.sample(full_dataset, SAMPLE_SIZE)
    print("Done")
    print(f"Loading mask, model output probabilites, and images into memory..")
    test_dataset = map(parse_image, test_dataset)# Creating dict linking images, true_mask, and pred_probs
    print("Done")
    #test_dataset = map(load_image_val, test_dataset)
    
    return test_dataset, SAMPLE_SIZE # a map iterator, int

In [10]:
def create_densecrf(crf_params: dict):
    """
    returns a densecrf() function with parameters from dict: crf_params
    called within objective function to allow crf to be modular
    
    pydensercrf: https://github.com/lucasb-eyer/pydensecrf
    
    Parameters
    ----------
    
    crf_params:
        # Except for 'gauss_scale_dims', all values are used in 'densecrf()'
        'gauss_compat': int # "strength of energy"
        'bilat_compat': int
        'bilat_scale_dims': (int, int) # horiz, vert standard deviation
        'gauss_scale_dims': (int, int) # used in test loop
        'bilat_scale_chans': (int, int, int) # RGB standard deviation
    """
    g_compat = crf_params['gauss_compat'] # strength
    bi_compat = crf_params['bilat_compat']
    bi_sdims = (crf_params['bilat_scale_dims'], crf_params['bilat_scale_dims'])
    bi_schans = (crf_params['bilat_scale_chans'], crf_params['bilat_scale_chans'], crf_params['bilat_scale_chans'])

    def densecrf(pred_mask, img, g_feats):
        img = np.reshape(img, (256, 256, 3))
        softmax = pred_mask.transpose((2, 0, 1))
        unary = utils.unary_from_softmax(softmax)
    
        # The inputs should be C-continious -- we are using Cython wrapper
        unary = np.ascontiguousarray(unary)
    
        d = dcrf.DenseCRF(img.shape[0] * img.shape[1], 2) # h * w, channels
        d.setUnaryEnergy(unary)
    
        #gaussian penalizes small pieces of segmentation that are spatially isolated
        d.addPairwiseEnergy(g_feats, compat=g_compat,
                    kernel=dcrf.DIAG_KERNEL,
                    normalization=dcrf.NORMALIZE_SYMMETRIC)
    
        # This creates the color-dependent features --
        # because the segmentation that we get from CNN are too coarse
        # and we can use local color features to refine them
        bi_feats = utils.create_pairwise_bilateral(sdims=bi_sdims,
                        schan=bi_schans, img=img, chdim=2)

        d.addPairwiseEnergy(bi_feats, compat=bi_compat,
                     kernel=dcrf.DIAG_KERNEL,
                     normalization=dcrf.NORMALIZE_SYMMETRIC)
    
        Q = d.inference(1) # could be tuned
        res = np.argmax(Q, axis=0).reshape((img.shape[0], img.shape[1]))
        return np.reshape(res, (256, 256, 1))
    
    return densecrf # returns function

In [11]:
def IOU(array, val_array):
    assert array.shape == val_array.shape, "Array and Validation Array have different sizes."
    np.reshape(array, (HEIGHT, WIDTH))
    np.reshape(val_array, (HEIGHT, WIDTH))
    intersection = np.sum(array == val_array)
    union = array.shape[0] * array.shape[1]
    res = intersection / union
    assert res <= 1
    return res

In [12]:
def objective(weights, crf_params: dict, test_dataset: iter , DATASET_SIZE: int):
    """
    objective function takes in crf_params and runs 
    testing on dataset from compile_dataset()
    
    Paramaters
    ----------
    
        crf_params: dict
            # Except for 'gauss_scale_dims', all values are used in 'densecrf()'
            'gauss_compat': int # "strength of energy"
            'bilat_compat': int
            'bilat_scale_dims': (int, int) # horiz, vert standard deviation
            'gauss_scale_dims': (int, int) # only used in objective func      
            'bilat_scale_chans': (int, int, int) # RGB standard deviation
        
        test_dataset: iter
             # map from list[dicts]:
            'image': np.array # (256, 256, 3) # image rgb data used by crf
            'true_mask': np.array # (256, 256, 3) # later trimmed to one channel image              
            'pred_probs': np.array # (256, 256, 2) # probs of two classes passed to crf
                
    Returns: mean_iou(float)
    """
    
    def probs_to_mask(pred_probs, densecrf, img, g_feats: int):
        pred_mask = densecrf(pred_probs, img, g_feats) # whats getting tuned
#         pred_mask = tf.argmax(pred_probs, axis=-1)
#         pred_mask = pred_mask[..., tf.newaxis]
        pred_mask = 1 - pred_mask
        pred_mask = pred_mask * 255
        return np.array(pred_mask).astype('uint8') # binary (0,255) values now
        
    
    AUTOTUNE = tf.data.experimental.AUTOTUNE
    #print(f"Tensorflow ver. {tf.__version__}")

    # For reproducibility
    SEED = 42
    BATCH_SIZE = 16
    BUFFER_SIZE = 1000
    
    densecrf = create_densecrf(crf_params) # crf function, for tuning!
    g_sdims = crf_params['gauss_scale_dims'] # another part of crf
    
    #Listing GPU info
    gpus = tf.config.experimental.list_physical_devices('GPU')
    if gpus:
       try:
           for gpu in gpus:
               tf.config.experimental.set_memory_growth(gpu, True)
           logical_gpus = tf.config.experimental.list_logical_devices('GPU')
           #print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
       except RuntimeError as e:
           print(e)
            
    model = sm.Unet(
        'vgg16', 
        #backbone,
        input_shape=(HEIGHT, WIDTH, 3), 
        encoder_weights='imagenet', 
        weights=weights,
        encoder_freeze=True,    # only training decoder network
        classes= 2, 
        activation='softmax'
    )

    # Might be unnecessary
    model.compile(
        'Adam', 
        #loss=sm.losses.bce_jaccard_loss, 
        loss=sm.losses.DiceLoss(),
        metrics=[sm.metrics.iou_score]
    )

    # CRF Pairwise energies:
    # This potential penalizes small pieces of segmentation that are
    # spatially isolated -- enforces more spatially consistent segmentations
    
    g_sdims = crf_params['gauss_scale_dims'] # scaling factor
    g_feats = utils.create_pairwise_gaussian(sdims=(g_sdims, g_sdims), shape=(HEIGHT, WIDTH))
    
    tot_iou = 0 # counter for mean_iou
    crf_time = 0 

    #print(f"The Tuning Dataset contains {DATASET_SIZE} images.")
    for np_dict in test_dataset:
        img = np_dict['image'] # (256, 256, 3) RGB 
        true_mask = np_dict['true_mask'][:, :, :1] # (256, 256, 3)->(256, 256, 1) grayscale        
        pred_probs = np_dict['pred_probs'] # (256, 256, 2) output probs
        
        start = time.time()
        # shape: (256, 256, 2) --densecrf-> (256, 256, 1)
        pred_mask = probs_to_mask(pred_probs, densecrf, img, g_feats) 
        crf_time += time.time() - start
        tot_iou += IOU(pred_mask, true_mask)
    
    mean_iou = tot_iou/DATASET_SIZE # objective value
#     print("Avg crf time: ", crf_time/DATASET_SIZE ) # display avg time spent int densecrf func
#     print("Mean IOU: ", mean_iou ) # display iou
    
    return -1 * mean_iou # optimization attempts to lower this number, so invert it 

In [13]:
def objective_wrapper(crf_params):
    weights = "../dataset/training/weights/08_10_vgg16_200_full_weight.h5"
    test_dataset = dataset
    test_size = size 
    return objective(weights, crf_params, test_dataset, size)

In [14]:
# test_crf_params = {
#     'gauss_compat': 3,
#     'bilat_compat': 3,
#     'gauss_scale_dims' : 3,  # only used in objective func      
#     'bilat_scale_dims' : 3, 
#     'bilat_scale_chans': 3
# }

In [29]:
%%time
global dataset
global size
dataset, size = compile_dataset(0.05)
dataset = list(dataset)

1 Physical GPUs, 1 Logical GPUs
The Tuning FULL Dataset contains 106560 images.
Taking sample of 5328 from main dataset..
Done
Loading mask, model output probabilites, and images into memory..
Done
CPU times: user 12.3 s, sys: 634 ms, total: 13 s
Wall time: 14.3 s


In [26]:
# generate prediction probabilites
weight_file = "../dataset/training/weights/08_10_vgg16_200_full_weight.h5"
img_dir = "../dataset/tuning/images/"
out_dir = "../dataset/tuning/probs/"

gen_pred_probs(img_dir, out_dir, weight_file)

1 Physical GPUs, 1 Logical GPUs


100%|██████████| 106560/106560 [43:23<00:00, 40.93it/s] 


In [31]:
dataset[0]

{'image': array([[[ 40, 102,  91],
         [ 42, 104,  93],
         [ 45, 107,  96],
         ...,
         [ 36, 107,  93],
         [ 40, 111,  97],
         [ 44, 115, 101]],
 
        [[ 42, 104,  93],
         [ 43, 105,  94],
         [ 45, 107,  96],
         ...,
         [ 38, 109,  95],
         [ 42, 113,  99],
         [ 44, 115, 101]],
 
        [[ 44, 106,  95],
         [ 44, 106,  95],
         [ 45, 107,  96],
         ...,
         [ 40, 111,  97],
         [ 42, 113,  99],
         [ 43, 114, 100]],
 
        ...,
 
        [[ 41, 104,  93],
         [ 39, 102,  91],
         [ 37, 100,  89],
         ...,
         [ 36, 107,  93],
         [ 37, 108,  94],
         [ 38, 109,  95]],
 
        [[ 40, 103,  92],
         [ 38, 101,  90],
         [ 37, 100,  89],
         ...,
         [ 35, 106,  92],
         [ 37, 108,  94],
         [ 39, 110,  96]],
 
        [[ 39, 102,  91],
         [ 38, 101,  90],
         [ 37, 100,  89],
         ...,
         [ 35, 106,

In [17]:
#objective(weight_file, test_crf_params, dataset, size)

In [27]:
# Create the domain space
test_space = {
    'gauss_compat': scope.int(hp.randint('gauss_compat', 10)),
    'bilat_compat': scope.int(hp.randint('bilat_compat', 10)),
    'gauss_scale_dims' : scope.int(hp.quniform('gauss_scale_dims', 1, 10, q=1)),  # only used in objective func      
    'bilat_scale_dims' : scope.int(hp.quniform('bilat_scale_dims', 1, 10, q=1)), 
    'bilat_scale_chans': scope.int(hp.quniform('bilat_scale_chans',1, 10, q=1)) 
}

In [28]:
# Create the algorithm
tpe_algo = tpe.suggest

# Create a trials object
tpe_trials = Trials()

In [ ]:
best = fmin(
    objective_wrapper,
    space=test_space,
    algo=tpe_algo,
    max_evals=50,
    trials=tpe_trials)

  2%|▏         | 1/50 [07:50<6:23:58, 470.18s/trial, best loss: -0.3004256526271144]

In [24]:
tpe_trials.results

[{'loss': -0.30891496641141875, 'status': 'ok'},
 {'loss': -0.30891417311476516, 'status': 'ok'},
 {'loss': -0.30891524420844185, 'status': 'ok'},
 {'loss': -0.3089151038779869, 'status': 'ok'},
 {'loss': -0.30891492058922937, 'status': 'ok'},
 {'loss': -0.30891493777255036, 'status': 'ok'},
 {'loss': -0.308915241344555, 'status': 'ok'},
 {'loss': -0.3089153129417259, 'status': 'ok'},
 {'loss': -0.30891492918088986, 'status': 'ok'},
 {'loss': -0.3089149062697952, 'status': 'ok'},
 {'loss': -0.3089152928945181, 'status': 'ok'},
 {'loss': -0.308914926317003, 'status': 'ok'},
 {'loss': -0.3089143535396358, 'status': 'ok'},
 {'loss': -0.30891521556957346, 'status': 'ok'},
 {'loss': -0.30891614060502154, 'status': 'ok'},
 {'loss': -0.30891494350032406, 'status': 'ok'},
 {'loss': -0.3089152470723287, 'status': 'ok'},
 {'loss': -0.3089140986537074, 'status': 'ok'},
 {'loss': -0.3089149177253425, 'status': 'ok'},
 {'loss': -0.30891485471983215, 'status': 'ok'},
 {'loss': -0.3089171229182063, '